In [166]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']
print len(estims_data)
#with open('./max/меланома_все.json') as data_file: #2
#with open('./max/рак_груди_все.json') as data_file: #0
with open('./max/рак_простаты_все.json') as data_file: #4
#with open('./max/рак_шейки_матки_все.json') as data_file: #1
#with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
print len(cancer_data)

10
230


In [153]:
FEATURES = [
 "views", 
# "large", 
# "top", 
# "price_level", 
# "speed_level", 
 "count_orders", 
# "count_orders_success", 
# "count_reviews", 
# "top_proc_cli", 
 "gen_proc_cli", 
# "top_oper_cli", 
# "gen_oper_cli", 
# "doc_count", 
# "doctor_prof_count", 
# "max_exp_doc", 
# "mean_exp_doc", 
# "max_h_index", 
# "mean_h_index", 
# "doc_top_diag", 
# "doc_gen_diag", 
# "doc_top_oper", 
# "doc_gen_oper", 
# "ill_cli", 
# "specialist", 
# "experience", 
 "variety", 
# "price"
]

In [110]:
clinic_ids = [i['id'] for i in estims_data]
#estims_data.pop(clinic_ids.index('0'))
#print sorted(clinic_ids)
#print sorted([i['id'] for i in cancer_data])

In [154]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    X+=[int(clinic['views'])]
    X+=[int(clinic['count_orders'])]
    X+=[len(all_proc)] #gen_proc_cli
#    X+=[int(clinic_estim['specialist'])]
#    X+=[int(clinic_estim['experience'])]
    X+=[int(clinic_estim['variety'])]
    y=int(clinic_estim['rating'])
    return [X, y]

In [112]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

In [113]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=False, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    #cv = cross_validation.ShuffleSplit(len(X)-1, n_iter=3,test_size=0.3, random_state=0)
    #scores = cross_validation.cross_val_score(regr, train_X, train_y, cv = 5)
    #print "scores"
    #print scores
    #print scores.mean()
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [155]:
X, y = get_X_y_from(cancer_data)

In [156]:
print X

[[24622   990    65     9]
 [ 5593   176    47    10]
 [27262   196    48     9]
 [ 4644   502    24     8]
 [ 9014   342    63     7]
 [19276   405    36     6]
 [ 4111    51    16     8]
 [18287   361    52     9]
 [22460   368    46     8]
 [ 6156   301    28     8]]


In [ ]:
for i in range(1,10):
    regr = process_with(X, y, info=True, ts=i/10.0)
    print

In [158]:
regr = process_with(X, y, info=True, ts=0.1)

Total: 10, train: 9, test: 1
Residual sum of squares: 0.65
Train absolute: 2.22
Test absolute: 0.81
Absolute to mean: 16.13%
Train variance score: 0.30
Test variance score: 0.00


In [159]:
import warnings
warnings.filterwarnings("ignore")
res= []
for i in range(0,len(y)):
    res += [(regr.predict(X[i]),y[i])]
    print "Real: %f \t Predicted: %f" %(y[i], regr.predict(X[i]))

Real: 10.000000 	 Predicted: 8.328249
Real: 9.000000 	 Predicted: 6.286692
Real: 8.000000 	 Predicted: 4.765252
Real: 7.000000 	 Predicted: 5.473319
Real: 6.000000 	 Predicted: 6.346430
Real: 5.000000 	 Predicted: 4.193315
Real: 4.000000 	 Predicted: 3.654009
Real: 3.000000 	 Predicted: 6.024662
Real: 2.000000 	 Predicted: 5.129720
Real: 1.000000 	 Predicted: 4.948528


In [160]:
sorted(res)
for i in sorted(res, reverse=True):
    print i[0][0], "\t", i[1]

8.32824902597 	10
6.34642987718 	6
6.2866917017 	9
6.02466220279 	3
5.47331865544 	7
5.12971960422 	2
4.94852821021 	1
4.76525216881 	8
4.19331457149 	5
3.65400893821 	4


In [136]:
plt.plot([i[0] for i in res])
plt.show()

In [162]:
print "Features sorted by their score:"
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
#f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
#print regr.intercept_
#print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0]

Features sorted by their score:
0.363800 	 variety
0.051400 	 gen_proc_cli
0.003200 	 count_orders
-0.000100 	 views


In [165]:
cancer_data[0]['doctors']

[{u'degree': u'\u041f\u0440\u043e\u0444\u0435\u0441\u0441\u043e\u0440',
  u'experience': u'20',
  u'h_index': u'17',
  u'id': u'1',
  u'illnesses': [{u'id': u'1',
    u'name_en': u'Esophageal cancer',
    u'name_ru': u'\u0420\u0430\u043a \u043f\u0438\u0449\u0435\u0432\u043e\u0434\u0430'},
   {u'id': u'3',
    u'name_en': u'Breast cancer',
    u'name_ru': u'\u0420\u0430\u043a \u0433\u0440\u0443\u0434\u0438'},
   {u'id': u'4',
    u'name_en': u'Lung cancer',
    u'name_ru': u'\u0420\u0430\u043a \u043b\u0435\u0433\u043a\u0438\u0445'},
   {u'id': u'7',
    u'name_en': u'Pancreatic cancer ',
    u'name_ru': u'\u0420\u0430\u043a \u043f\u043e\u0434\u0436\u0435\u043b\u0443\u0434\u043e\u0447\u043d\u043e\u0439 \u0436\u0435\u043b\u0435\u0437\u044b'},
   {u'id': u'13',
    u'name_en': u'Vitiligo',
    u'name_ru': u'\u0412\u0438\u0442\u0438\u043b\u0438\u0433\u043e'},
   {u'id': u'51',
    u'name_en': u'Stomach cancer',
    u'name_ru': u'\u0420\u0430\u043a \u0436\u0435\u043b\u0443\u0434\u043a\u0430'

In [106]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

4
3
5
6
2
1
